# Overview
1. Create dataframe from S&P 500 wiki table
2. Create dataframe from S&P 500 stock analysis table
3. Combine the two dataframes
4. Create a CSV that can be referenced for the enitery of the quarter

# Beautiful Soup Approach

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Get the page's HTML content
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(URL)

# Convert the table to a Pandas DataFrame
df = pd.read_html(response.content, flavor='lxml')[0]

# Display the first few rows of the table
display(df)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


### Find random company

In [10]:
import random
N = len(df)
for i in range(100):
    pick_random_company = random.randint(0,N-1)
    if pick_random_company > 500:
        print(pick_random_company)
        
print(pick_random_company)
display(df.iloc[pick_random_company])

493


Symbol                                    WY
Security                        Weyerhaeuser
GICS Sector                      Real Estate
GICS Sub-Industry               Timber REITs
Headquarters Location    Seattle, Washington
Date added                        1979-10-01
CIK                                   106535
Founded                                 1900
Name: 493, dtype: object

### Get wiki link for each company & append it to the dataframe in a new column

In [20]:
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('table', {'class': 'wikitable'})


# Find all of the links in the second column of the table
links = []
for row in table.find_all('tr')[1:]: # skip the header row
    cells = row.find_all('td')
    if cells:
        link_tag = cells[1].find('a')  # Adjust the index based on the column containing the link
        if link_tag and 'href' in link_tag.attrs:
            links.append(link_tag['href'])

# Create column in existing dataframe with link as element, but with full url
updated_df = df.assign(link=links)
updated_df['wiki_link'] = 'https://en.wikipedia.org' + updated_df['link']
display(updated_df.keys())

# save the dataframe to a CSV file
# updated_df.to_csv('S&P_500_companies.csv', index=False)



Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded', 'link',
       'wiki_link'],
      dtype='object')

# Get revenue, market cap, and stock price from other website
https://stockanalysis.com/list/sp-500-stocks/

In [12]:
import pandas as pd
new_url = "https://stockanalysis.com/list/sp-500-stocks/"
new_response = requests.get(new_url)
# soup = BeautifulSoup(new_response.text, 'html.parser')

# Find the table
# table = soup.find('table', {'id': 'main-table'})

new_df = pd.read_html(new_response.content, flavor='lxml')[0]
display(new_df)



,No.,Symbol,Company Name,Market Cap,Stock Price,% Change,Revenue
0,1,AAPL,Apple Inc.,"3,301.73B",217.16,-1.70%,385.60B
1,2,MSFT,Microsoft Corporation,"3,071.40B",413.21,1.85%,245.12B
2,3,NVDA,NVIDIA Corporation,"2,633.79B",107.37,0.85%,96.31B
3,4,AMZN,"Amazon.com, Inc.","1,872.20B",178.38,1.70%,604.33B
4,5,GOOGL,Alphabet Inc.,"1,848.22B",150.14,0.96%,328.28B
...,...,...,...,...,...,...,...
498,499,AAL,American Airlines Group Inc.,7.25B,11.05,-1.54%,53.45B
499,500,IVZ,Invesco Ltd.,7.10B,15.78,-0.97%,5.81B
500,501,PARA,Paramount Global,6.82B,10.23,-0.82%,29.27B
501,502,ETSY,"Etsy, Inc.",6.05B,52.71,0.69%,2.77B


# Join the two dataframes to get a new dataframe that contains all of the necessary information

In [21]:
df_joined = pd.merge(updated_df, new_df[["Symbol", "Market Cap", "Stock Price", "Revenue"]], on='Symbol')
# display(df_joined)

null_mask = df_joined.isnull().any(axis=1)
null_rows = df_joined[null_mask]

print(null_rows)

Empty DataFrame
Columns: [Symbol, Security, GICS Sector, GICS Sub-Industry, Headquarters Location, Date added, CIK, Founded, link, wiki_link, Market Cap, Stock Price, Revenue]
Index: []


In [ ]:
def pick_random_company(sp500_df: pd.DataFrame) -> dict:
    """_summary_

    Args:
        sp500_df (pd.DataFrame): dataframe containing the s&p 500 companies and their respective information

    Returns:
        dict: attributes of the randomly selected company
    """
    N = len(sp500_df)
    for i in range(100):
        random_digit = random.randint(0,N-1)
    
    random_company = {
        "Name":sp500_df.iloc[random_digit]["Security"], 
        "Sector":sp500_df.iloc[random_digit]["GICS Sector"], 
        "Founded":sp500_df.iloc[random_digit]["Founded"], 
        "Headquarters":sp500_df.iloc[random_digit]["Headquarters Location"]
        }
    
    return random_company